# Displaying Sensor Data on the LED Bar

A key concept in computers is taking an input from a sensor, processing that data in a meaningful way, and outputting that to the user in a format that is intuitive and easy to read. This concept can be found in almost every computing system. 

We are going to create this type of system with our sensors and LED bar. The end product will be a sort of "digital thermometer".

### This notebook will walk you through using the TH02 sensor

If your sensor looks like this, it is a TH02 and you should follow this notebook. If not, you should use the ATH20 sensor notebook [here](temp_ledbar_example_ATH20.ipynb).

![TH02](https://files.seeedstudio.com/wiki/Grove-TemptureAndHumidity_Sensor-High-Accuracy_AndMini-v1.0/img/main.jpg)

As usual, apply the base overlay to the board:

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# 1. Read Temperature Data

### Import Libraries

In [2]:
from pynq.lib.arduino import Grove_TH02
from pynq.lib.arduino import ARDUINO_GROVE_I2C

### Plugging in your TH02 Sensor

This sensor uses the Arduino shield to communicate with your PYNQ Board. Attach the Arduino shield to the top of your PYNQ Board. Then, plug in the TH02 Sensor to one of the ports labeled I2C on the Arduino shield.

![grove_adaptor](https://cdn10.bigcommerce.com/s-7gavg/products/531/images/4194/PYNQ_Shield_-_Oblique_-_600__81639.1473444236.1280.1280.png?c=2)

### Initialize the Temperature Sensor

We create an variable called "th02" that represents the sensor. From this object we can call functions that retrieve data from the sensor.

In [3]:
th02 = Grove_TH02(base.ARDUINO,ARDUINO_GROVE_I2C)

### Read the Temperature and Humidity
Calling the function read() returns a tuple (two values separated by a comma). The first value is temperature and the second value is humidity.

In [4]:
temp_c, humidity = th02.read()
print('The temperature is {} degrees celsius and the relative humidity is {} %'.format(temp_c, humidity))

The temperature is 28.21875 degrees celsius and the relative humidity is 40.0625 %


<a id='ath'></a>

# Display the Data to the LED Bar

## Connecting the LED Bar

Plug the LED bar into the port labeled G1 on your PMOD to Grove Adaptor. Then plug the PMOD to Grove adaptor into the port labeled PMODB on your PYNQ-Z2 Board

### Setup

When the hardware is connected, run the following code to tell the PYNQ Board how you connected everything. If you plugged into a port besides G1, such as G2, then you will need to import `PMOD_GROVE_G2` instead.

In [5]:
from pynq.lib.pmod import Grove_LEDbar
from pynq.lib.pmod import PMOD_GROVE_G1

ledbar = Grove_LEDbar(base.PMODB, PMOD_GROVE_G1)

### Reset the LED Bar

The default state of the LED could be all sorts of things. Let's turn it fully of so we have somewhere to work from.

In [6]:
# Turn the LED bar off
ledbar.reset()

### Decide on Your Temperature Scale

We want to make a visual thermometer that displays the temperature across a range that we care about. Perhaps this is simply comfortable room temperature (somewhere between 60 and 80 degrees fahrenheit), or it's the temperature of industrial equipment that cannot get above 150 degrees fahrenheit. What we will do next is establish this range and "map" it to the 10 bars on the LED indicator, with the red being the highest end of the range and the green being the lowest. For this example, lets the temperature we might find in a house in Colorado, 60 to 80 degrees fahrenheit, or about 15 to 25 degrees celsius (remember, our sensors read celsius, so make sure to convert when necessary)

In [16]:
max_temp = 25
min_temp = 15
temp_range = max_temp - min_temp

We should consider "edge cases" where our temperature falls above or below our range. These are always good to check before we proceed. In this situation, we will illuminate all LEDs if the temperature is above the range, and 1 LED if it is below the range.

Once we have ruled out edge cases, we can deal with the case where our temperature falls within our allotted range. Because there are 10 bars, we can only only display the temperature in tenths of our range. Knowing what our range is, we can calculate a number between 0 and 10 to represent where within the range we fall. Note that we multiply by 10 and round to the nearest whole number to make sure we pass an integer between 0 and 10 to the function up_to().

Because our LED bar speaks in binary, We are going to use a function "up_to" which produces the binary value required to light up the bars "up to" a certain index on the led bar, given that index. This is covered more in depth in the [LED Bar notebook](../bootcamp_sessions/3_led_bar.ipynb).

In [17]:
def up_to(index):
    return ~(2**(10 - index) - 1)

In [18]:
# if temperature is above our range of measurement
if temp_c > max_temp:
    ledbar.write_binary(up_to(10))
# if temperature is below our range of measurement
elif temp_c < min_temp:
    ledbar.write_binary(up_to(0))
# otherwise is falls within our range
else:
    temp_index = int(round(((temp_c - min_temp)/temp_range)*10))
    ledbar.write_binary(up_to(temp_index))